In [1]:
import subprocess
import tkinter as tk
from tkinter import ttk

def get_wifi_profiles():
    try:
        data = subprocess.check_output(['netsh', 'wlan', 'show', 'profiles']).decode('utf-8').split('\n')
        profiles = [line.split(":")[1].strip() for line in data if "All User Profile" in line]
        return profiles
    except subprocess.CalledProcessError as e:
        print("Error:", e)
        return []

def get_wifi_password(profile):
    try:
        results = subprocess.check_output(['netsh', 'wlan', 'show', 'profile', profile, 'key=clear']).decode('utf-8').split('\n')
        password_lines = [line.split(":")[1].strip() for line in results if "Key Content" in line]
        return password_lines[0] if password_lines else None
    except subprocess.CalledProcessError as e:
        print("Error:", e)
        return None

def show_passwords():
    profile_list.delete(0, tk.END)
    profiles = get_wifi_profiles()
    for profile in profiles:
        password = get_wifi_password(profile)
        profile_list.insert(tk.END, f"{profile}: {password if password else 'Password not available'}")

# Create the main window
root = tk.Tk()
root.title("Wi-Fi Password Viewer")

# Create a frame for the listbox
frame = ttk.Frame(root, padding="10")
frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Create a listbox to display profiles and passwords
profile_list = tk.Listbox(frame, width=50, height=15)
profile_list.grid(row=0, column=0, padx=5, pady=5)

# Create a button to trigger the display of passwords
show_button = ttk.Button(frame, text="Show Wi-Fi Passwords", command=show_passwords)
show_button.grid(row=1, column=0, pady=5)

# Create a scrollbar for the listbox
scrollbar = ttk.Scrollbar(frame, orient=tk.VERTICAL, command=profile_list.yview)
scrollbar.grid(row=0, column=1, sticky=(tk.N, tk.S))
profile_list.config(yscrollcommand=scrollbar.set)

root.mainloop()
